In [83]:
import numpy as np
import pandas as pd
import copy

from hongxiongmao import download
from hongxiongmao import overplot
dl = download.quandl_hxm()

import plotly.offline as py
import plotly_express as pyex
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

In [84]:
cli = dl.from_tickerdict('oecd_cli', start_date='-20y') - 100
US = cli.loc[:,'US']

In [249]:
def swirlygram(df, roc=3, trail=18, **kwargs):
    """
    """
    
    ### Default Setup
    cmap=overplot.COLOURMAP.copy()
    layout=copy.deepcopy(overplot.DEFAULT_LAYOUT)
    
    ### Manipulate Data
    df = df.iloc[-(trail+roc):,:]
    x = (df - df.shift(roc)).iloc[-trail:,:]
    y = df.iloc[-trail:,:]
    
    ### Append Data
    data=[]
    for i, c in enumerate(df.columns):

        # LINE
        data.append(dict(type='scatter', name=c, mode='lines+markers', showlegend=True,
                    x=x.loc[:,c], y=y.loc[:,c],
                    line=dict(color=cmap[i], width=1),))
            
        # MARKER for most recent observation
        data.append(dict(type='scatter', mode='markers', showlegend=False, 
                    x=[x.iloc[-1,:].loc[c]], y=[y.iloc[-1,:].loc[c]],
                    marker=dict(symbol='diamond', color=cmap[i], size=10,
                               line={'color':'black', 'width':1}),))
    
        ## Buttons
        
        if i == 0:l = len(data)    # find no traces on first pass
        
        visible = [False] * l * len(df.columns)    # List of False = Total No of Traces
        visible[(i*l):(i*l)+l] = [True] * l        # Make traces of ith pass visible
        
        button= dict(label=c, method='update', args=[{'visible': visible},
                                                     {'marker':dict(color='black')}])
        layout['updatemenus'][0]['buttons'].append(button)     # append the button 
    
    ### Additional Layout Changes
    
    layout=overplot._update_layout(layout, **kwargs)
    
    # Symmetrical Plot around zero
    absmax = lambda df, x=1: np.max([df.abs().max().max(), x]) * 1.05    
    layout['xaxis1']['range'] = [-absmax(x), absmax(x)]
    layout['yaxis1']['range'] = [-absmax(y), absmax(y)]
    
    # Rectangles of colour for each quadrant
    shapes = [{'type':'rect', # Top Right
               'xref':'x', 'x0':0, 'x1':100,
               'yref':'y', 'y0':0, 'y1':100,
               'line':{'width': 0,},'fillcolor':'green', 'opacity': 0.1,},    
              {'type':'rect', # Bottom Left
               'xref':'x', 'x0':-100, 'x1':0, 
               'yref':'y', 'y0':-100, 'y1':0,
               'line':{'width': 0,}, 'fillcolor':'red', 'opacity': 0.1,},    
              {'type':'rect', # Top Left
               'xref':'x', 'x0':-100, 'x1':0,
               'yref':'y', 'y0':0, 'y1':100, 
               'line':{'width': 0,}, 'fillcolor':'blue', 'opacity': 0.1,},
              {'type': 'rect', # Bottom Right
               'xref':'x', 'x0':0, 'x1':100,
               'yref':'y', 'y0':-100, 'y1':0,
               'line':{'width': 0,}, 'fillcolor':'orange', 'opacity': 0.1,},]
    
    layout['shapes'] = shapes
    
    fig = dict(data=data, layout=layout)
    
    return fig

fig = swirlygram(cli.iloc[:,0:8], title='OECD Normalised CLI Swirlygram',
                 yaxis={'title':'some shit'},
                 button_position=[1.1, 0.6], button_direction = 'down')
py.iplot(go.Figure(fig))

In [ ]:
def swirlygram(df, roc=3, trail=12, animations=True):
    """
    """
    
    ### Default Setup
    cmap=overplot.COLOURMAP.copy()
    layout=copy.deepcopy(overplot.DEFAULT_LAYOUT)
    data=[]
    
        
    # Data for the Cth country
    x, y = (df.iloc[:,0] - df.iloc[:,0].shift(roc)), df.iloc[:,0]
        
    ## Leading Indicator
    
    # LINE
    data.append(dict(type='scatter', name='', mode='lines+markers', showlegend=True,
                     x=df.iloc[-trail:,0], y=df.iloc[-trail:,0],
                     line=dict(color=cmap[0], width=1),
                     #visible=True if i==1 else False,
                     ))
            
    # MARKER for most recent observation
    data.append(dict(type='scatter', mode='markers', showlegend=False, 
                     x=[x.iloc[-1]],
                     y=[y.iloc[-1]],
                     marker=dict(symbol='diamond', color=cmap[0], size=15),
                     # visible=True if j==0 else False,
                     ))
        
    frames=[]
    sliders = {'yanchor': 'top', 'xanchor': 'left', 
                           'currentvalue': {'font':{'size': 10}, 'prefix':'Component Vals & Correlation Date: ', 'xanchor': 'left'},
                           'transition': {'duration': 10, 'easing': 'linear'},
                           'pad': {'b': 0, 't': 25}, 'len': 1, 'x': 0, 'y': 0,
                           'steps':[]}
    
    fig = dict(data=data, layout=layout)
    
    #for i, v in enumerate(x):
        
    #    label = str(i)
        
     #   frames.append({'name':i,
      #                 'layout':{},
      #                 'data':[dict(type='scatter', mode='markers', x=[x.iloc[-i]] , y=[y.iloc[-i]], xaxis='x1', yaxis='y1',),
      #                         dict(type='scatter', mode='lines+markers', x=[x.iloc[-i-trail:i]] , y=[y.iloc[-i-trail:i]], xaxis='x1', yaxis='y1',),
      #                ]})
      #  
      #  sliders['steps'].append({'label':label, 'method': 'animate', 
      #                                   'args':[[i], {'frame': {'duration': 0, 'easing':'linear', 'redraw': True},
      #                                          'transition':{'duration': 0, 'easing': 'linear'}}],
      #                                  })
      #  
      #  fig['frames'] = frames
      #  fig['layout']['sliders'] = [sliders]
    
    return fig

fig = swirlygram(cli.iloc[-24:,0:1])
py.iplot(fig)